# Create MXNet Container

In [ ]:
# Create the binary
!git clone https://github.com/aws/sagemaker-mxnet-container.git
!cd sagemaker-mxnet-container
!git checkout v3.1.2
!python setup.py sdist

In [ ]:
# MXNet 1.4.1, Python 3, CPU
!cp dist/sagemaker_mxnet_container-3.1.2.tar.gz docker/1.4.1/py3/sagemaker_mxnet_container.tar.gz
!cd docker/1.4.1/py3/
!docker build -t preprod-mxnet:1.4.1-cpu-py3 -f Dockerfile.cpu .

In [ ]:
!cd ../../../
!pip install -e .[test]

In [ ]:
# May fail
!tox test/unit

In [ ]:
# Create an IAM role called [SageMakerRole] with AmazonSageMakerFullAccess and AmazonEC2ContainerRegistryFullAccess manually
# Add the IAM role of the notebook with AmazonEC2ContainerRegistryFullAccess mannually

In [ ]:
!docker images

In [ ]:
# May fail
!tox -- test/integration/local --docker-base-name preprod-mxnet \
                              --tag 1.4.1-cpu-py3 \
                              --py-version 3 \
                              --framework-version 1.4.1 \
                              --processor cpu

In [4]:
# upload container to ECS
# create-repository in ECR mannually
!$(aws ecr get-login --region ap-northeast-1 --no-include-email)
!docker login -u AWS -p = https://579019700964.dkr.ecr.ap-northeast-1.amazonaws.com
!docker tag preprod-mxnet:1.4.1-cpu-py3 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet:1.4.1-cpu-py3
!docker push 579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet:1.4.1-cpu-py3

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: login attempt to https://579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/v2/ failed with status: 400 Bad Request
The push refers to repository [579019700964.dkr.ecr.ap-northeast-1.amazonaws.com/preprod-mxnet]

c6cf436c: Preparing 
a47b956c: Preparing 
0ac446f2: Preparing 
09efe46f: Preparing 
368178b1: Preparing 
7ba6b896: Preparing 
fb893158: Preparing 
42719515: Preparing 
103e78c9: Preparing 
e637fbff: Preparing 
fb893158: Pushed   668.8MB/654.1MB1A1.4.1-cpu-py3: digest: sha256:9e388fdb9a291d4fc51adc682761477c677cc676ed22a28de240626e1f7365cb size: 2622


In [5]:
# May fail
!tox -- test/integration/sagemaker --aws-id 579019700964 \
                                  --docker-base-name preprod-mxnet \
                                  --instance-type ml.m4.xlarge \
                                  --tag 1.4.1-cpu-py3

/bin/sh: tox: command not found


# Create TextClassification Docker

In [ ]:
!cd gluon-nlp/model_zoo/text_classification/
# create train
!chmod +x train

# !wget https://raw.githubusercontent.com/facebookresearch/fastText/master/classification-results.sh
# !chmod +x classification-results.sh
# !./classification-results.sh
# !mv data/yelp_review_polarity.train .
# !mv data/yelp_review_polarity.test .
# !rm -rf data result

!aws s3 cp --recursive s3://app-mia/mytask/ .

# create hyperparameters.json
# create inputdataconfig.json
# create Dockerfile.train
!docker build -t text_classification_train -f Dockerfile.train .
!docker run text_classification_train train